<a href="https://colab.research.google.com/github/sbuttler/CTGAN/blob/master/CTGAN_newsvendor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade git+git://github.com/sbuttler/CTGAN.git

  Cloning git://github.com/sbuttler/CTGAN.git to /tmp/pip-req-build-x3agto68
  Running command git clone -q git://github.com/sbuttler/CTGAN.git /tmp/pip-req-build-x3agto68
     |████████████████████████████████| 10.4MB 2.8MB/s 
  Created wheel for ctgan: filename=ctgan-0.2.2.dev0-py2.py3-none-any.whl size=16895 sha256=f0e90d295d0f9b20bf401d505125695912cb20e6bf8c68b868a190ea322d2959
  Stored in directory: /tmp/pip-ephem-wheel-cache-16s15h6x/wheels/bd/ca/1b/42c278860383ea89e8773b12b0789d226d3b1c5c29921fc76c
Successfully built ctgan
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [1]:
!pip install pulp

     |████████████████████████████████| 40.6MB 81kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=7476491fce040d4482079d328d3287b104c54a5591b0c09e1317138e42ca110a
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


In [17]:
import pandas as pd
import pickle
import numpy as np

import itertools
import timeit

import ctgan.synthesizer as ctgan
from sklearn.preprocessing import MinMaxScaler

import torch

#from ddop.datasets.load_datasets import load_data
#from ddop.newsvendor import EmpiricalRiskMinimizationNewsvendor
#from ddop.metrics.costs import calc_costs, calc_total_costs, calc_avg_costs

from sklearn.model_selection import train_test_split
import lightgbm as lgb

import pulp

import multiprocessing as mp
from functools import partial
import itertools

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
model_path = '/content/drive/My Drive/GANs/'

In [8]:
demand = 'demand'

train = pd.read_pickle('/content/drive/My Drive/GANs/Data/20200629_train_FilNr2_ArtNr110_all_cols')
train.drop(columns=['Filiale', 'Artikel', 'WGruppe', 'time_index'], axis=1, inplace=True)
X_train = train.drop(columns=[demand])
Y_train = train[demand]

val = pd.read_pickle('/content/drive/My Drive/GANs/Data/20200629_val_FilNr2_ArtNr110_all_cols')
val.drop(columns=['Filiale', 'Artikel', 'WGruppe', 'time_index'], axis=1, inplace=True)
X_val = val.drop(columns=[demand])
Y_val = val[demand]

test = pd.read_pickle('/content/drive/My Drive/GANs/Data/20200629_test_FilNr2_ArtNr110_all_cols')
test.drop(columns=['Filiale', 'Artikel', 'WGruppe', 'time_index'], axis=1, inplace=True)
X_test = test.drop(columns=[demand])
Y_test = test[demand]

scaler = MinMaxScaler()
scaler.fit(train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [49]:
# Define dataframe to store solution

service_levels = [x/100 for x in range(5, 100, 5)]

result = pd.DataFrame(index=service_levels, columns=['cost_val_solution', 'cost_test_solution', 'GAN_model_parameters'])

result['cost_val_solution'] = np.inf
result['cost_test_solution'] = np.inf

# GAN parameters

In [10]:
epochs = [5]
#epochs = [5, 10, 50, 100]
batchsize = [30, 60, 130]
learning_rate = [1e-5, 5e-5, 1e-6]

training_repetition = 1

# Function definitions

In [11]:
def solve_newsvendor(demand_data, weights, sl):
    # set service level
    c_u = sl
    c_o = 1 - sl
    
    print('started newsvendor')

    sample_index = np.arange(demand_data.shape[0])
    
    demand = dict(zip(sample_index, demand_data))
    weights = dict(zip(sample_index, weights))

    m = pulp.LpProblem(sense=pulp.LpMinimize)

    # add variables
    q = pulp.LpVariable('q', lowBound=0)

    u = pulp.LpVariable.dicts('u', sample_index, lowBound=0)
    o = pulp.LpVariable.dicts('o', sample_index, lowBound=0)


    # set objective

    m += (sum(c_u*u[i]*weights[i] for i in sample_index) + sum(c_o * o[i]*weights[i] for i in sample_index))

    # add constraints
    for i in sample_index:

      m += u[i] >= demand[i] - q
      m += o[i] >= q - demand[i]
    
    m.solve()
    
    return q.value()

In [21]:
def compute_lightGBM_wSAA_weights(sl, gbm, X_train, Y_train, val_x):
    
    print("compute light GBM wSAA")
    # Predict test data
    predLeafs = gbm.predict(val_x, pred_leaf=True)

    # Calculate weights based on training data
    predLeafsTrain = gbm.predict(X_train, pred_leaf=True)
    elemInLeafs = np.sum(predLeafsTrain == predLeafs, axis=0)
    treeWeights = (predLeafsTrain == predLeafs)/elemInLeafs
    weights = np.sum(treeWeights, axis=1)/predLeafs.shape[1]
    
    # Optimize
    q = solve_newsvendor(Y_train, weights, sl)

    return q

In [13]:
def compute_lightGBM_wSAA(sl, train_X, train_Y, eval_X, eval_Y, test_eval_X):
    
    
    print('fit lightGBM model')
    
    # fit lightGBM model
    trainingData = lgb.Dataset(train_X, train_Y)
    evalData = lgb.Dataset(eval_X, eval_Y)

    param = {'objective': 'regression'}

    num_round = 1000

    # fitting model 
    gbm = lgb.train(
        param,
        trainingData,
        num_round,
        valid_sets=evalData,
        early_stopping_rounds=10,
        verbose_eval=100
    )
    
       
    wSAA_function = partial(compute_lightGBM_wSAA_weights, sl, gbm, train_X, train_Y)
    
    q = test_eval_X.apply(wSAA_function, axis=1)
    
    return q

In [14]:
def newsvendor_total_cost(y_pred, y_true, sl):
    
    total_cost = 0
    
    # set service level
    c_u = sl
    c_o = 1 - sl
    
    for i in range(y_pred.shape[0]):
        
        if y_pred[i] > y_true[i]:
            
            total_cost += (y_pred[i]-y_true[i])*c_o
            
        if y_pred[i] < y_true[i]:
            
            total_cost += (y_true[i]-y_pred[i])*c_u
    
    return total_cost

In [28]:
def lightGBM_wSAA_cost(gan_params, train_X, train_Y, eval_X, eval_Y, test_eval_X, test_eval_Y, sl):
    
    y_pred = compute_lightGBM_wSAA(sl, train_X, train_Y, eval_X, eval_Y, test_eval_X)
    
    y_true = test_eval_Y

    y_pred.reset_index(inplace=True, drop=True)
    y_true.reset_index(inplace=True, drop=True)
    
    return (sl, [newsvendor_total_cost(y_pred, y_true, sl), gan_params])

# Test with one GAN and small sample size

In [ ]:
service_levels = [0.7]

e = 5
bs = 30
lr = 1e-5
i = 1

p_gan = str(i)+"_e"+str(e)+"_b"+str(bs)+"_l"+str(lr)

gan = torch.load(model_path+"ctgan_model_"+str(i)+"_e"+str(e)+"_b"+str(bs)+"_l"+str(lr))

print('sample')
sample_sc = gan.sample(1000000, seed=0)
sample = pd.DataFrame(scaler.inverse_transform(sample_sc), columns=sample_sc.columns)

print('finished')
X = sample.drop(columns=[demand])
Y = sample[demand]

X_train_gan, X_val_gan, Y_train_gan, Y_val_gan = train_test_split(X, Y, test_size=0.2, random_state=99)
        
pool_function = partial(lightGBM_wSAA_cost, p_gan, X_train_gan, Y_train_gan,  X_val_gan, Y_val_gan, X_val, Y_val)

print('open pool')
#agents = mp.cpu_count()
agents = len(service_levels)
pool = mp.Pool(agents)

pool_output = pool.map(pool_function, service_levels)

pool.close()

pool_output = dict(pool_output)

print(pool_output)

for sl in service_levels:
      
  val_cost = pool_output.get(sl)[0]
  parameters = pool_output.get(sl)[1]
  
  if val_cost < result.loc[sl,'cost_val_solution']:
        
    result.loc[sl,'cost_val_solution'] = val_cost
    result.loc[sl, 'GAN_model_parameters'] = parameters
    print('test')


sample


In [51]:
result

,cost_val_solution,cost_test_solution,GAN_model_parameters
0.05,inf,inf,NaN
0.10,inf,inf,NaN
0.15,inf,inf,NaN
0.20,inf,inf,NaN
0.25,2610.218700,inf,1_e5_b30_l1e-05
0.30,inf,inf,NaN
0.35,inf,inf,NaN
0.40,inf,inf,NaN
0.45,inf,inf,NaN
0.50,4212.283619,inf,1_e5_b30_l1e-05


# Full GAN newsvendor loop

In [ ]:
for lr in learning_rate:

  for bs in batchsize:

    for e in epochs:

      for i in range(1,training_repetition+1):
        
        p_gan = (lr,bs,e,i)
        
        gan = torch.load(model_path+"ctgan_model_"+str(i)+"_e"+str(e)+"_b"+str(bs)+"_l"+str(lr))
        
        print('sample')
        sample_sc = gan.sample(1000000, seed=0)
        sample = pd.DataFrame(scaler.inverse_transform(sample_sc), columns=sample_sc.columns)
        
        print('finished')
        X = sample.drop(columns=[demand])
        Y = sample[demand]
        
        X_train_gan, X_val_gan, Y_train_gan, Y_val_gan = train_test_split(X, Y, test_size=0.2, random_state=99)
                
        pool_function = partial(ERM_cost, p_gan, X_train_gan, Y_train_gan, X_val, Y_val)

        print('open pool')
        agents = mp.cpu_count()
        pool = mp.Pool(agents)
        
        pool_output = pool.map(pool_function, service_levels)

        pool.close()

        pool_output = dict(pool_output)
        
        for sl in service_levels:
      
          val_cost = pool_output.get(sl)[0]
          parameters = pool_output.get(sl)[1]
          
          if val_cost < result.loc[sl,'cost_val_solution']:
                
            result.loc[sl,'cost_val_solution'] = val_cost
            result.loc[sl, 'GAN_model_parameters'] = parameters
            print('test')
